## Visualization techniques for scalar fields in VTK + Python
## Goals 
* Inspect VTK Objects via the qtconsole for Jupyter (using the magic `%qtconsole`)
* Including a new filter, mapper, and actor to visualize the complete/partial mesh 
* Computing new data arrays using the `vtkCalculator()` 
* Performing data transformations: from rectilinear grid to unstructured grid and image data
* Data filtering
* Visualizing scalar fields using, points, surfaces, isosurfaces, and volume rendering 
    * Basics of transfer functions
    

# Challenge 1: Adding a new Filter+Mapper+Actor to visualize the grid 
Try to find out how to visualize the mesh structure (grid). Take a look at [RectilinearGrid.py](http://www.vtk.org/Wiki/VTK/Examples/Python/RectilinearGrid/vtkRectilinearGrid) example from the VTK wiki.

In [1]:
import vtk
#help(vtk.vtkRectilinearGridReader())

rectGridReader = vtk.vtkRectilinearGridReader()
rectGridReader.SetFileName("data/jet4_0.500.vtk")
# do not forget to call "Update()" at the end of the reader
rectGridReader.Update()

In [2]:
rectGridOutline = vtk.vtkRectilinearGridOutlineFilter()
rectGridOutline.SetInputData(rectGridReader.GetOutput())

# New vtkRectilinearGridGeometryFilter() goes here:
plane = vtk.vtkRectilinearGridGeometryFilter()
plane.SetInputData(rectGridReader.GetOutput())
plane.SetExtent(0, 128, 0, 0, 0, 128)

rectGridOutlineMapper = vtk.vtkPolyDataMapper()
rectGridOutlineMapper.SetInputConnection(rectGridOutline.GetOutputPort())

rectGridGeomMapper = vtk.vtkPolyDataMapper()
rectGridGeomMapper.SetInputConnection(plane.GetOutputPort())

outlineActor = vtk.vtkActor()
outlineActor.SetMapper(rectGridOutlineMapper)
outlineActor.GetProperty().SetColor(0, 0, 0)

gridGeomActor = vtk.vtkActor()
gridGeomActor.SetMapper(rectGridGeomMapper)
# Find out how to visualize this as a wireframe 
gridGeomActor.GetProperty().SetRepresentationToWireframe()
# Play with the options you get for setting up actor properties (color, opacity, etc.)
gridGeomActor.GetProperty().SetColor(0, 0, 0)

# Create the usual rendering stuff.
renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
renderer.AddActor(gridGeomActor)
renderer.AddActor(outlineActor)
renderer.SetBackground(1, 1, 1)
 
renWin.SetSize(300, 300)
 
# interact with data
renWin.Render()
iren.Start()

# Challenge 2: Using the vtkCalulator to compute the vector magnitude
As you should have noticed our data set has only one point data array named vectors. We need now to use this array to calculate the magnitude of the vectors at each point of the grid. We will do this by using the [vtk.vtkArrayCalculator()](http://www.vtk.org/doc/nightly/html/classvtkArrayCalculator.html).

In [3]:
magnitudeCalcFilter = vtk.vtkArrayCalculator()
magnitudeCalcFilter.SetInputConnection(rectGridReader.GetOutputPort())
magnitudeCalcFilter.AddVectorArrayName('vectors')
# Set up here the array that is going to be used for the computation ('vectors')
magnitudeCalcFilter.SetResultArrayName('magnitude')
magnitudeCalcFilter.SetFunction("mag(vectors)")
# Set up here the function that calculates the magnitude of a vector
magnitudeCalcFilter.Update()

#Inspect the output of the calculator using the IPython console to verify the result
print(magnitudeCalcFilter.GetOutput())

vtkRectilinearGrid (00000000041E9370)
  Debug: Off
  Modified Time: 12690292
  Reference Count: 2
  Registered Events: (none)
  Information: 0000000006E777A0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 12690293
  Field Data:
    Debug: Off
    Modified Time: 12690271
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 4194304
  Number Of Cells: 4112895
  Cell Data:
    Debug: Off
    Modified Time: 12690279
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0000000006E301D0)
          Event: 33
          EventName: ModifiedEvent
          Command: 0000000006CBB800
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 )
    Scalars: (none)
   

# Challenge 3: Visualize the data set as colored points based on the "magnitude" value
Take a look at the [`subset`](http://www.vtk.org/doc/nightly/html/classvtkMaskPoints.html) object. What happens when you change the SetOnRadio parameter? Try out changing between different `RandomModeType` options. What does the `SetScalarModeToUsePointData()` function does?

In [4]:
#Extract the data from the result of the vtkCalculator
points = vtk.vtkPoints()
grid = magnitudeCalcFilter.GetOutput()
grid.GetPoints(points)
scalars = grid.GetPointData().GetArray('magnitude')

#Create an unstructured grid that will contain the points and scalars data
ugrid = vtk.vtkUnstructuredGrid()
ugrid.SetPoints(points)
ugrid.GetPointData().SetScalars(scalars)

#Populate the cells in the unstructured grid using the output of the vtkCalculator
for i in range (0, grid.GetNumberOfCells()):
    cell = grid.GetCell(i)
    ugrid.InsertNextCell(cell.GetCellType(), cell.GetPointIds())

In [5]:
#There are too many points, let's filter the points
subset = vtk.vtkMaskPoints()
subset.SetOnRatio(50)
subset.RandomModeOn()
subset.SetInputData(ugrid)

#Make a vtkPolyData with a vertex on each point.
pointsGlyph = vtk.vtkVertexGlyphFilter()
pointsGlyph.SetInputConnection(subset.GetOutputPort())
#pointsGlyph.SetInputData(ugrid)
pointsGlyph.Update()

pointsMapper = vtk.vtkPolyDataMapper()
pointsMapper.SetInputConnection(pointsGlyph.GetOutputPort())
pointsMapper.SetScalarModeToUsePointData()

pointsActor = vtk.vtkActor()
pointsActor.SetMapper(pointsMapper)

# Challenge 4: Visualize the data set as isosurfaces based on the "magnitude" value
Go to the documentation of vtkContourFilter and explain what does the GenerateValues() do?  Inspect the scalarRange of the magnitude array. What happens when you change these values in the function? 

In [6]:
scalarRange = ugrid.GetPointData().GetScalars().GetRange()
lut = vtk.vtkLookupTable()
lut.SetNumberOfColors(256)
lut.SetHueRange(0.667, 0.0)
lut.SetVectorModeToMagnitude()
lut.Build()
isoFilter = vtk.vtkContourFilter()
isoFilter.SetInputData(ugrid)
isoFilter.GenerateValues(10, scalarRange)

isoMapper = vtk.vtkPolyDataMapper()
isoMapper.SetInputConnection(isoFilter.GetOutputPort())
isoMapper.SetLookupTable(lut)

isoActor = vtk.vtkActor()
isoActor.SetMapper(isoMapper)
isoActor.GetProperty().SetOpacity(0.5)

# An alternative to define colormaps 

In [36]:
lut = vtk.vtkLookupTable()
lut.SetNumberOfColors(256)
lut.SetHueRange(0.667, 0.0)
lut.SetVectorModeToMagnitude()
lut.Build()

hh = vtk.vtkHedgeHog()
hh.SetInputConnection(subset.GetOutputPort())
hh.SetScaleFactor(0.001)

hhm = vtk.vtkPolyDataMapper()
hhm.SetInputConnection(hh.GetOutputPort())
hhm.SetLookupTable(lut)
hhm.SetScalarVisibility(True)
hhm.SetScalarModeToUsePointFieldData()
hhm.SelectColorArray('vectors')
hhm.SetScalarRange((grid.GetOutput().GetPointData().GetVectors().GetRange(-1)))

AttributeError: 'vtkCommonDataModelPython.vtkRectilinearGrid' object has no attribute 'GetOutput'

### Renderer, render window, and interactor 

In [ ]:
#Option 1: Default vtk render window
renderer = vtk.vtkRenderer()
renderer.SetBackground(0.5, 0.5, 0.5)
renderer.AddActor(outlineActor)
renderer.AddActor(isoActor)
renderer.ResetCamera()

renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindow.SetSize(500, 500)
renderWindow.Render()

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renderWindow)
iren.Start()